<div style="font-size:20px"><p>
Tensor Processing Units (TPUs) are Google’s custom-developed application-specific integrated circuits (ASICs) used to accelerate machine learning workloads previous algorithm take took weeks to train on GPUs can even be trained in hours using TPUs.</p><p> We can use TPUs from cloud services like google cloud or for free from Kaggle or google colab In this tutorial we will see how can we use TPU on TFRecords. TFRecords is a file format optimized for TensorFlow that is used for storing a sequence of binary records. It is very useful for large dataset since only data that is required is loaded in batches</p>
</div>

# Importing Libraries

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import tensorflow as tf
import keras
import keras.layers as L

In [ ]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print("Device:", tpu.master())
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
except:
    strategy = tf.distribute.get_strategy()
print("Number of replicas:", strategy.num_replicas_in_sync)

To use TPU, we must load our data from GCS to that we need GCS path of our dataset which we can get using KaggleDatasets

In [ ]:
from kaggle_datasets import KaggleDatasets

In [ ]:
AUTO = tf.data.experimental.AUTOTUNE
GCS_DS_Path = KaggleDatasets().get_gcs_path('tpu-getting-started')
print(GCS_DS_Path)

In [ ]:
IMAGE_SIZE = [224,224]
GCS_PATH = GCS_DS_Path + '/tfrecords-jpeg-224x224'

In [ ]:
training_file = tf.io.gfile.glob(GCS_PATH+'/train/*.tfrec') 
test_file = tf.io.gfile.glob(GCS_PATH+'/test/*.tfrec')
valid_file = tf.io.gfile.glob(GCS_PATH+'/val/*.tfrec')

# Preparing Data

To make TFRecords dataset we will use TFRecordDataset Class available in tf.data module

In [ ]:
def decode_image(image_data):
    image = tf.image.decode_jpeg(image_data, channels=3)
    image = tf.cast(image, tf.float32) / 255.0 
    image = tf.reshape(image, [*IMAGE_SIZE, 3])
    return image

def read_labeled_tfrecord(example):
    LABELED_TFREC_FORMAT = {
        "image": tf.io.FixedLenFeature([], tf.string), # tf.string means bytestring
        "class": tf.io.FixedLenFeature([], tf.int64),  # shape [] means single element
    }
    example = tf.io.parse_single_example(example, LABELED_TFREC_FORMAT)
    image = decode_image(example['image'])
    label = tf.cast(example['class'], tf.int32)
    return image, label # returns a dataset of (image, label) pairs
def read_unlabeled_tfrecord(example):
    UNLABELED_TFREC_FORMAT = {
        "image": tf.io.FixedLenFeature([], tf.string), # tf.string means bytestring
        "id": tf.io.FixedLenFeature([], tf.string),  # shape [] means single element
        # class is missing, this competitions's challenge is to predict flower classes for the test dataset
    }
    example = tf.io.parse_single_example(example, UNLABELED_TFREC_FORMAT)
    image = decode_image(example['image'])
    idnum = example['id']
    return image, idnum # returns a dataset of image(s)

def load_dataset(filenames, labeled=True, ordered=False):
    # Read from TFRecords. For optimal performance, reading from multiple files at once and
    # disregarding data order. Order does not matter since we will be shuffling the data anyway.

    ignore_order = tf.data.Options()
    if not ordered:
        ignore_order.experimental_deterministic = False # disable order, increase speed

    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads=AUTO) # automatically interleaves reads from multiple files
    dataset = dataset.with_options(ignore_order) # uses data as soon as it streams in, rather than in its original order
    dataset = dataset.map(read_labeled_tfrecord if labeled else read_unlabeled_tfrecord, num_parallel_calls=AUTO)
    # returns a dataset of (image, label) pairs if labeled=True or (image, id) pairs if labeled=False
    return dataset

In [ ]:

def get_training_dataset():
    dataset = load_dataset(training_file, labeled=True)
    dataset = dataset.repeat() # the training dataset must repeat for several epochs
    dataset = dataset.shuffle(2048)
    dataset = dataset.batch(BATCH_SIZE)
    return dataset

def get_validation_dataset(ordered=False):
    dataset = load_dataset(valid_file, labeled=True, ordered=ordered)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.cache()
    return dataset

def get_test_dataset(ordered=False):
    dataset = load_dataset(test_file, labeled=False, ordered=ordered)
    dataset = dataset.batch(BATCH_SIZE)
    return dataset

In [ ]:
BATCH_SIZE = 16 * strategy.num_replicas_in_sync

ds_train = get_training_dataset()
ds_valid = get_validation_dataset()
ds_test = get_test_dataset()

In [ ]:
ds_iter = iter(ds_train.unbatch().batch(20))

In [ ]:
one_batch = next(ds_iter)

In [ ]:
for i in range(1,20):
    plt.subplot(4,5,i)
    plt.imshow(one_batch[0][i],aspect='auto')

# Model

<p style="font-size:18px">VGG16 is a convolutional neural network model proposed by K. Simonyan and A. Zisserman from the University of Oxford in the paper “Very Deep Convolutional Networks for Large-Scale Image Recognition”. The model achieves 92.7% top-5 test accuracy in ImageNet, which is a dataset of over 14 million images belonging to 1000 classes.</p>

<img src="https://miro.medium.com/max/2268/1*CrjJwSX9S7f759dK2EtGJQ.png">

Helper Function to make ConvBlock

In [ ]:
def convblock(filter_size,is_block2=False):
    model.add(L.Conv2D(filter_size,kernel_size=(3,3),padding='same',activation='relu'))
    model.add(L.Conv2D(filter_size,kernel_size=(3,3),padding='same',activation='relu'))
    if is_block2:
        model.add(L.Conv2D(filter_size,kernel_size=(3,3),padding='same',activation='relu'))
    model.add(L.MaxPool2D(pool_size=(2,2),strides=(2,2),padding='same'))

In [ ]:
weights = keras.utils.get_file('vgg16_weights','https://storage.googleapis.com/tensorflow/keras-applications/vgg16/vgg16_weights_tf_dim_ordering_tf_kernels.h5')

In [ ]:
with strategy.scope():
    model = keras.Sequential()
    model.add(L.InputLayer(input_shape=(224,224,3)))
    convblock(64)
    
    convblock(128)
    
    convblock(256,is_block2=True)

    convblock(512,is_block2=True)
    
    convblock(512,is_block2=True)
    model.add(L.Flatten())
    model.add(L.Dense(4096,activation='relu'))
    model.add(L.Dense(4096,activation='relu'))
    model.add(L.Dense(1000,activation='relu'))
    model.load_weights(weights)
    for Layers in model.layers:
        Layers.trainable = False
    model.add(L.Dense(104, activation='softmax')) # since our dataset have 104 classes

In [ ]:
model.summary()

In [ ]:
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss = 'sparse_categorical_crossentropy',
    metrics=['sparse_categorical_accuracy'],
)

In [ ]:
NUM_TRAINING_IMAGES = 12753
NUM_TEST_IMAGES = 7382
STEPS_PER_EPOCH = NUM_TRAINING_IMAGES // BATCH_SIZE

In [ ]:
history = model.fit(
    ds_train,
    validation_data=ds_valid,
    epochs=50,steps_per_epoch=STEPS_PER_EPOCH
)

In [ ]:
test_ds = get_test_dataset(ordered=True)

print('Computing predictions...')
test_images_ds = test_ds.map(lambda image, idnum: image)
probabilities = model.predict(test_images_ds)
predictions = np.argmax(probabilities, axis=-1)
print(predictions)

In [ ]:
print('Generating submission.csv file...')

# Get image ids from test set and convert to unicode
test_ids_ds = test_ds.map(lambda image, idnum: idnum).unbatch()
test_ids = next(iter(test_ids_ds.batch(NUM_TEST_IMAGES))).numpy().astype('U')

# Write the submission file
np.savetxt(
    'submission.csv',
    np.rec.fromarrays([test_ids, predictions]),
    fmt=['%s', '%d'],
    delimiter=',',
    header='id,label',
    comments='',
)

# Look at the first few predictions
!head submission.csv

# Refrences
1. )https://www.kaggle.com/ryanholbrook/create-your-first-submission
2. )https://neurohive.io/en/popular-networks/vgg16/
3. )https://cloud.google.com/tpu
4. )https://en.wikipedia.org/wiki/Tensor_Processing_Unit
5. )https://medium.com/mostly-ai/tensorflow-records-what-they-are-and-how-to-use-them-c46bc4bbb564 